# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [40]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [98]:
# response --> trying to access the URL and GET the information from the web site.
response = requests.get(url)

# Our response contains all the information from the URL (source code). 
# By converting it to text, it's easier to parse the information within
data = response.text


## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [42]:
from bs4 import BeautifulSoup

In [107]:
# Passing source code of web page to BeautifulSoup. This way we create a BeautifulSoup object.
soup = BeautifulSoup(data, features='html.parser')

# Extracting all the <a> tags contained in the source code of the URL and putting them in a list.
# The <a> tag defines a hyperlink, which is used to link from one page to another.
# The most important attribute of the <a> element is the href attribute, which indicates the link's destination.
tags = soup.find_all('a', {"href": True})

# Using set comprehension to create a set and eliminate duplicates.
unique_urls = {tag.get('href') for tag in tags}


## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [50]:
domain = 'http://wikipedia.org'

In [124]:
abs_links = [link for link in unique_urls if '%' not in link and 'http' in link]
len(abs_links)


58

In [123]:
rel_links = [domain + link for link in unique_urls if '%' not in link and 'http' not in link and not link.startswith('#')]
len(rel_links)


220

In [72]:
total_links = abs_links + rel_links
len(set(total_links))

278

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [75]:
import os

In [80]:
# Using os.getcwd() to check the folder we're in, and also to check the path the new folder will follow.
path = os.getcwd() + '/wikipedia'
os.mkdir(path)

In [83]:
# os.chdir changes the cwd to the path we give it.
os.chdir(path)
os.getcwd()

'/Users/soul/Desktop/IronHack/lab-parallelization/your-code/wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [84]:
from slugify import slugify

In [108]:
def index_page(link):
    response = requests.get(link).text
    if response:
        filename = slugify(link) + '.html'
        with open(filename, 'w+') as f:
            f.write(response)
    else:
        pass
            

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [131]:
%%time
for link in total_links:
   index_page(link)


CPU times: user 5 µs, sys: 7 µs, total: 12 µs
Wall time: 16.9 µs


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [126]:
import multiprocessing

In [128]:
pool = multiprocessing.Pool()

In [130]:
%%time
result = pool.map(index_page, total_links)
pool.close()
pool.join()
